In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import warnings

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from scipy.stats import rankdata, pearsonr

import shap

In [ ]:
def reduce_mem_usage(dataframe, dataset):
    """
    Reduces the memory footprint of a DataFrame by downcasting numeric columns 
    to the most efficient data types that can safely hold the data.

    This function iterates over each column in the DataFrame and attempts to:
    - Downcast integer columns to the smallest possible int subtype (int8, int16, etc.).
    - Downcast float columns to the smallest possible float subtype (float16, float32, etc.).
    - Skip non-numeric columns.

    It prints the memory usage before and after optimization, along with the percentage reduction.

    Args:
        dataframe (pd.DataFrame): The input DataFrame to optimize.
        dataset (str): A label or name of the dataset for logging purposes.

    Returns:
        pd.DataFrame: The optimized DataFrame with reduced memory usage.
    """
    print('Reducing memory usage for:', dataset)
    initial_mem_usage = dataframe.memory_usage().sum() / 1024**2
    
    for col in dataframe.columns:
        col_type = dataframe[col].dtype

        c_min = dataframe[col].min()
        c_max = dataframe[col].max()
        if str(col_type)[:3] == 'int':
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                dataframe[col] = dataframe[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                dataframe[col] = dataframe[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                dataframe[col] = dataframe[col].astype(np.int32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                dataframe[col] = dataframe[col].astype(np.int64)
        else:
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                dataframe[col] = dataframe[col].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                dataframe[col] = dataframe[col].astype(np.float32)
            else:
                dataframe[col] = dataframe[col].astype(np.float64)

    final_mem_usage = dataframe.memory_usage().sum() / 1024**2
    print('--- Memory usage before: {:.2f} MB'.format(initial_mem_usage))
    print('--- Memory usage after: {:.2f} MB'.format(final_mem_usage))
    print('--- Decreased memory usage by {:.1f}%\n'.format(100 * (initial_mem_usage - final_mem_usage) / initial_mem_usage))

    return dataframe

In [ ]:
def add_features(df):
    df = df.copy()
    df['bid_ask_spread'] = df['ask_qty'] - df['bid_qty']
    df['bid_ask_ratio'] = df['bid_qty'] / (df['ask_qty'] + 1e-8)
    df['total_liquidity'] = df['bid_qty'] + df['ask_qty']
    df['liquidity_imbalance'] = (df['bid_qty'] - df['ask_qty']) / (df['total_liquidity'] + 1e-8)
    df['normalized_spread'] = df['bid_ask_spread'] / (df['total_liquidity'] + 1e-8)
    
    df['buy_sell_ratio'] = df['buy_qty'] / (df['sell_qty'] + 1e-8)
    df['net_order_flow'] = df['buy_qty'] - df['sell_qty']
    df['order_flow_imbalance'] = df['net_order_flow'] / (df['buy_qty'] + df['sell_qty'] + 1e-8)
    df['volume_participation'] = df['volume'] / (df['buy_qty'] + df['sell_qty'] + 1e-8)
    df['aggressive_ratio'] = (df['buy_qty'] + df['sell_qty']) / (df['volume'] + 1e-8)
    
    df['buy_pressure'] = df['buy_qty'] / (df['volume'] + 1e-8)
    df['sell_pressure'] = df['sell_qty'] / (df['volume'] + 1e-8)
    df['net_pressure'] = df['buy_pressure'] - df['sell_pressure']
    df['pressure_ratio'] = df['buy_pressure'] / (df['sell_pressure'] + 1e-8)
    
    df['depth_ratio'] = df['total_liquidity'] / (df['volume'] + 1e-8)
    df['bid_depth_ratio'] = df['bid_qty'] / (df['volume'] + 1e-8)
    df['ask_depth_ratio'] = df['ask_qty'] / (df['volume'] + 1e-8)
    df['depth_imbalance'] = (df['bid_depth_ratio'] - df['ask_depth_ratio']) / (df['depth_ratio'] + 1e-8)
    
    df['kyle_lambda'] = np.abs(df['net_order_flow']) / (df['volume'] + 1e-8)
    df['amihud_illiquidity'] = np.abs(df['net_pressure']) / (df['volume'] + 1e-8)
    df['liquidity_consumption'] = df['volume'] / (df['total_liquidity'] + 1e-8)
    
    df['price_efficiency'] = 1 / (1 + df['amihud_illiquidity'])
    df['execution_quality'] = df['volume'] / (df['bid_ask_spread'] + 1)
    
    df['pin_proxy'] = np.abs(df['order_flow_imbalance']) * df['amihud_illiquidity']
    df['order_toxicity'] = np.abs(df['order_flow_imbalance']) * df['kyle_lambda']
    
    df['bid_momentum'] = df['bid_qty'] * df['buy_qty'] / (df['volume'] + 1e-8)
    df['ask_momentum'] = df['ask_qty'] * df['sell_qty'] / (df['volume'] + 1e-8)
    df['liquidity_adjusted_volume'] = df['volume'] / np.sqrt(df['total_liquidity'] + 1)
    
    df['log_volume'] = np.log1p(df['volume'])
    df['log_liquidity'] = np.log1p(df['total_liquidity'])
    df['log_spread'] = np.log1p(np.abs(df['bid_ask_spread']))

    # Replace any NaNs or Infs
    df = df.replace([np.inf, -np.inf], 0).fillna(0)
    
    return df


In [ ]:
def add_statistical_features(df):
    """
    Adds statistical aggregation features across all 'X'-prefixed columns:
    - Mean
    - Std Dev
    - Range (max - min)
    - Median
    - 25th percentile
    - 75th percentile
    - Count of values above row mean
    - Index of max and min column (numeric suffix)
    - Cleans up NaNs and infs
    """
    x_cols = [col for col in df.columns if col.startswith('X') and col[1:].isdigit()]
    x_data = df[x_cols]

    # Core stats
    df['x_stat_mean'] = x_data.mean(axis=1)
    df['x_stat_std'] = x_data.std(axis=1)
    df['x_stat_range'] = x_data.max(axis=1) - x_data.min(axis=1)
    df['x_stat_median'] = x_data.median(axis=1)
    df['x_stat_p25'] = x_data.quantile(0.25, axis=1)
    df['x_stat_p75'] = x_data.quantile(0.75, axis=1)

    # Count of values above row mean
    row_means = df['x_stat_mean'].values[:, None]
    df['x_stat_above_mean_count'] = (x_data.values > row_means).sum(axis=1)

    # Index (suffix) of max and min column
    df['x_stat_idx_max'] = x_data.idxmax(axis=1).str.extract(r'(\d+)', expand=False).astype(float)
    df['x_stat_idx_min'] = x_data.idxmin(axis=1).str.extract(r'(\d+)', expand=False).astype(float)

    # Cleanup: handle infs and NaNs
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)

    return df
    

In [ ]:
def add_non_linear_x_market_interactions(df):
    """
    Adds non-linear interaction features between 'X'-prefixed columns and selected market features.

    - For each column starting with 'X' followed by digits (e.g., 'X1', 'X23'), creates new features by 
      multiplying the X-column with the logarithm of each available market feature.
    - Market features considered: volume, buy_qty, sell_qty, x_stat_mean, x_stat_median.
    - Only uses market features that are present in the DataFrame.
    - Resulting feature name format: '{X_col}_log_x_{market_feature}'.
    - Cleans the resulting DataFrame by replacing NaNs and infinities with zero.

    Returns:
        pd.DataFrame: The input DataFrame with new interaction features added.
    """
    market_features = ['volume', 'buy_qty', 'sell_qty','x_stat_mean', 'x_stat_median']
    x_cols = [col for col in df.columns if col.startswith('X') and col[1:].isdigit()]
    
    # Filter only available market features
    market_features = [feat for feat in market_features if feat in df.columns]

    for x_col in x_cols:
        for m_feat in market_features:
            new_col = f'{x_col}_log_x_{m_feat}'
            df[new_col] = df[x_col] * np.log(df[m_feat])
    
    # Cleanup
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)

    return df

In [ ]:
def select_top_k_features_by_shap(X_train, y_train, feature_names, k=30):
    """Select top-k features using mean absolute SHAP value importance"""
    print("Calculating SHAP-based feature importance...")

    # Convert categories to int
    X_train = X_train.copy()
    for col in X_train.select_dtypes(include='category').columns:
        X_train[col] = X_train[col].astype(int)

    # Train XGBoost model
    params = {
        'n_estimators': 100,
        'max_depth': 6,
        'learning_rate': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': 0
    }

    model = XGBRegressor(**params)
    model.fit(X_train, y_train)

    # Compute SHAP values
    explainer = shap.Explainer(model, X_train)
    shap_values = explainer(X_train)

    # Calculate mean absolute SHAP value per feature
    mean_abs_shap = np.abs(shap_values.values).mean(axis=0)
    shap_importance_df = pd.DataFrame({
        'feature': X_train.columns,
        'importance': mean_abs_shap
    }).sort_values('importance', ascending=False)

    # Select top-k features
    selected_features = shap_importance_df.head(k)['feature'].tolist()

    # Always include critical features
    critical_features = ['order_flow_imbalance', 'kyle_lambda', 'vpin', 'volume',
                         'bid_ask_spread', 'liquidity_imbalance', 'buying_pressure']

    for feat in critical_features:
        if feat in feature_names and feat not in selected_features:
            selected_features.append(feat)

    print(f"Selected top {k} SHAP features (plus critical ones if needed). Total: {len(selected_features)}")

    return selected_features, shap_importance_df



In [ ]:
def create_time_decay_weights(n, decay=0.95):
    """
    Generates a sequence of time-decay weights for a series of length `n`.

    Weights are assigned such that more recent observations (towards the end of the series)
    receive higher weight, following an exponential decay pattern. The weights are normalized
    to sum to `n`, preserving the original scale.

    Args:
        n (int): Number of observations.
        decay (float, optional): Decay rate between 0 and 1. Lower values decay faster.
                                 Defaults to 0.95.

    Returns:
        np.ndarray: A 1D array of shape (n,) containing the normalized time-decay weights.
    """
    pos = np.arange(n)
    norm = pos / (n - 1)
    w = decay ** (1.0 - norm)
    return w * n / w.sum()

def adjust_weights_for_outliers(X, y, base_weights, outlier_fraction=0.001):
    """
    Adjusts sample weights to downweight outliers based on model prediction residuals.

    A quick Random Forest model is trained using the provided features and labels
    to estimate residuals. Samples with the largest residuals (as defined by 
    `outlier_fraction`) are considered outliers and receive lower weights.

    The adjustment scales down the base weights for outliers on a linear scale 
    between 0.2 and 0.8 of the original value, depending on residual severity.

    Args:
        X (pd.DataFrame or np.ndarray): Feature matrix.
        y (pd.Series or np.ndarray): Target values.
        base_weights (np.ndarray): Original sample weights.
        outlier_fraction (float, optional): Fraction of samples to treat as outliers.
                                            Defaults to 0.001.

    Returns:
        np.ndarray: Adjusted sample weights with reduced influence for outliers.
    """
    # Train quick model to estimate residuals
    model = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1)
    model.fit(X, y, sample_weight=base_weights)
    preds = model.predict(X)
    residuals = np.abs(y - preds)

    # Top N residuals = outliers
    n_outliers = max(1, int(outlier_fraction * len(residuals)))
    threshold = np.partition(residuals, -n_outliers)[-n_outliers]
    outlier_mask = residuals >= threshold

    # Downweight outliers (linear scale: 0.2–0.8 of base weight)
    adjusted_weights = base_weights.copy()
    if outlier_mask.any():
        res_out = residuals[outlier_mask]
        res_norm = (res_out - res_out.min()) / (res_out.ptp() + 1e-8)
        weight_factors = 0.8 - 0.6 * res_norm
        adjusted_weights[outlier_mask] *= weight_factors

    return adjusted_weights


In [ ]:
warnings.filterwarnings('ignore')

train = pd.read_parquet("/kaggle/input/drw-crypto-market-prediction/train.parquet")
test = pd.read_parquet("/kaggle/input/drw-crypto-market-prediction/test.parquet")

In [ ]:
base_features = list(dict.fromkeys([
    "X752", "X287", "X298", "X759", "X302", "X55", "X56", "X52", "X303", "X51",
    "X598", "X385", "X603", "X674", "X415", "X345", "X174", "X178", "X168", "X612",
    "bid_qty", "ask_qty", "buy_qty", "sell_qty", "volume",
    "X758", "X296", "X611", "X780", "X451", "X25", "X591", "X727", "X427", "X288",
    "X721", "X312", "X421", "X471", "X573", "X255", "X144", "X299", "X301", "X563",
    "X737", "X702", "X507", "X306", "X501", "X586", "X43", "X517", "X248", "X137",
    "X757", "X196", "X777", "X280", "X266", "X689", "X294", "X492", "X555", "X731",
    "X262", "X576", "X13", "X518", "X502", "X558", "X6", "X602", "X695", "X703",
    "X413", "X660", "X37", "X15", "X310", "X512", "X362", "X631", "X214", "X562",
    "X488", "X510", "X256", "X35", "X128", "X86", "X170", "X30", "X265", "X323",
    "X559", "X348", "X130", "X529", "X20", "X4", "X90", "X192", "X91", "X582", "X99",
    "X24", "X317", "X707", "X653", "X519", "X557", "X371", "X84", "X83", "X360",
    "X111", "X699", "X187", "X637", "X567", "X577", "X313", "X60", "X671", "X698",
    "X701", "X725", "X292", "X638", "X741", "X379", "X700", "X614", "X676", "X516",
    "X697", "X311", "X615", "X706", "X466", "X571", "X17", "X584", "X436", "X305",
    "X34", "X282", "X681", "X7", "X208", "X41", "X536", "X548", "X776", "X87", "X40",
    "X570", "X539", "X474", "X753", "X425", "X217", "X199", "X18", "X609", "X21",
    "X277", "X279", "X326", "X540", "X688", "X553", "X452", "X738", "X183",
    "label"
]))

In [ ]:
train = reduce_mem_usage(train, "train")
test = reduce_mem_usage(test, "test")

train = train[base_features]
test = test[base_features]

train = train.dropna().reset_index(drop=True)
test = test.fillna(0)

In [ ]:
x_train = add_features(train)
x_train = add_statistical_features(x_train)
x_train = add_non_linear_x_market_interactions(x_train)

In [ ]:
x_test = add_features(test)
x_test = add_statistical_features(x_test)
x_test = add_non_linear_x_market_interactions(x_test)
x_train = x_train[np.isfinite(x_train['label'])].reset_index(drop=True)

In [ ]:
# Ensure the index is a datetime type
x_train.index = pd.to_datetime(x_train.index)

# Sort the dataset by timestamp (index)
x_train = x_train.sort_index()

# Final training features and labels (no validation split)
X_train = x_train.drop(columns=['label'])
y_train = x_train['label']

In [ ]:
#feature_names = X_train.columns.tolist()
#selected_features, importance_df = select_top_k_features_by_shap(X_train, y_train, feature_names, k=95)

selected_features = ['X451', 'X758', 'X452', 'X345', 'X752', 'X425', 'X466', 'X757', 'X178', 'X137', 'X759', 'X174', 'X777', 'X611', 'X752_log_x_buy_qty', 'X303', 'X128', 'X40', 'X757_log_x_volume', 'X282', 'X280', 'X385', 'X759_log_x_x_stat_median', 'X780', 'X752_log_x_volume', 'X20', 'X584', 'X292', 'X501', 'x_stat_p75', 'X612', 'X752_log_x_sell_qty', 'X415_log_x_buy_qty', 'X24', 'X326', 'X738', 'X421', 'X178_log_x_volume', 'X759_log_x_sell_qty', 'X296', 'X287', 'X427', 'X21', 'X759_log_x_buy_qty', 'X586', 'X570', 'X507', 'X91', 'X591', 'X35', 'X37', 'X302', 'X753_log_x_buy_qty', 'X614_log_x_volume', 'X452_log_x_volume', 'X345_log_x_buy_qty', 'X298', 'X413', 'X576', 'X415', 'X415_log_x_volume', 'X757_log_x_sell_qty', 'X37_log_x_sell_qty', 'X466_log_x_volume', 'X653', 'X614', 'X602', 'X425_log_x_volume', 'X757_log_x_buy_qty', 'X87', 'X425_log_x_buy_qty', 'X130', 'X282_log_x_volume', 'X18', 'X582', 'X753_log_x_sell_qty', 'X287_log_x_sell_qty', 'X577', 'X256', 'X776', 'X288', 'X137_log_x_volume', 'X21_log_x_x_stat_median', 'X279', 'X385_log_x_volume', 'X752_log_x_x_stat_mean', 'X178_log_x_buy_qty', 'X84', 'X674', 'X611_log_x_volume', 'X299', 'X345_log_x_x_stat_mean', 'X529', 'X277_log_x_buy_qty', 'X90', 'order_flow_imbalance', 'kyle_lambda', 'bid_ask_spread', 'liquidity_imbalance']

In [ ]:
X_train = X_train[selected_features]
X_test = x_test[selected_features]

In [ ]:
X_train

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
import warnings
warnings.filterwarnings('ignore')

def simple_prophet_outlier_detection(df, feature_col, timestamp_col='__index_level_0__'):
    """
    Simple example of using Prophet to detect outliers in a single feature.
    
    The idea: Prophet models the expected behavior of the feature over time.
    Points that fall far outside Prophet's confidence interval are outliers.
    """
    print(f"Detecting outliers in {feature_col} using Prophet...")
    
    # 1. Prepare data for Prophet
    prophet_df = pd.DataFrame({
        'ds': df[timestamp_col],
        'y': df[feature_col]
    })
    
    # Remove obvious bad values
    prophet_df = prophet_df[np.isfinite(prophet_df['y'])]
    
    # Resample to hourly for efficiency (adjust based on your needs)
    prophet_hourly = prophet_df.set_index('ds').resample('1H').mean().reset_index()
    prophet_hourly = prophet_hourly.dropna()
    
    # 2. Fit Prophet model
    model = Prophet(
        changepoint_prior_scale=0.05,  # Low value = less sensitive to outliers
        interval_width=0.95,  # 95% confidence interval
        yearly_seasonality=False,
        weekly_seasonality=True,
        daily_seasonality=True
    )
    
    model.fit(prophet_hourly)
    
    # 3. Generate predictions
    forecast = model.predict(prophet_hourly)
    
    # 4. Identify outliers
    # Method 1: Points outside confidence interval
    outliers_ci = (
        (prophet_hourly['y'] < forecast['yhat_lower']) | 
        (prophet_hourly['y'] > forecast['yhat_upper'])
    )
    
    # Method 2: Points with large standardized residuals
    residuals = prophet_hourly['y'] - forecast['yhat']
    residual_std = residuals.std()
    outliers_residual = np.abs(residuals) > 3 * residual_std
    
    # Combine both methods
    outliers = outliers_ci & outliers_residual
    
    # 5. Visualize
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Plot 1: Time series with outliers
    ax1.plot(prophet_hourly['ds'], prophet_hourly['y'], 'b.', alpha=0.5, label='Actual')
    ax1.plot(forecast['ds'], forecast['yhat'], 'g-', linewidth=2, label='Prophet Fit')
    ax1.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], 
                     alpha=0.2, color='green', label='95% CI')
    
    # Highlight outliers
    outlier_points = prophet_hourly[outliers]
    ax1.scatter(outlier_points['ds'], outlier_points['y'], 
               color='red', s=100, edgecolor='darkred', linewidth=2, 
               label=f'Outliers ({outliers.sum()})', zorder=10)
    
    ax1.set_title(f'Prophet Outlier Detection: {feature_col}')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Value')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Residual distribution
    ax2.hist(residuals, bins=50, alpha=0.7, color='blue', edgecolor='black')
    ax2.axvline(x=-3*residual_std, color='red', linestyle='--', label='±3σ threshold')
    ax2.axvline(x=3*residual_std, color='red', linestyle='--')
    ax2.set_title('Residual Distribution')
    ax2.set_xlabel('Residual (Actual - Predicted)')
    ax2.set_ylabel('Frequency')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 6. Return outlier information
    outlier_info = {
        'n_outliers': outliers.sum(),
        'outlier_pct': outliers.sum() / len(prophet_hourly) * 100,
        'outlier_timestamps': outlier_points['ds'].tolist(),
        'outlier_values': outlier_points['y'].tolist(),
        'residual_std': residual_std
    }
    
    print(f"Found {outlier_info['n_outliers']} outliers ({outlier_info['outlier_pct']:.2f}%)")
    
    return outlier_info, model, forecast


# Quick example for multiple features
def detect_outliers_multiple_features(data_path, features_to_check=None):
    """
    Run outlier detection on multiple features.
    """
    # # Load data
    # if features_to_check is None:
    #     # Default to some common features
    #     features_to_check = ['volume', 'bid_qty', 'ask_qty', 'buy_qty', 'sell_qty']
    
    # df = pd.read_parquet(data_path, columns=['__index_level_0__'] + features_to_check)
    
    # Ensure timestamp
    if '__index_level_0__' not in df.columns:
        df['__index_level_0__'] = pd.date_range('2023-03-01', periods=len(df), freq='T')
    
    # Analyze each feature
    outlier_summary = {}
    
    for feature in features_to_check:
        if feature in df.columns:
            print(f"\n{'='*60}")
            outlier_info, model, forecast = simple_prophet_outlier_detection(df, feature)
            outlier_summary[feature] = outlier_info
    
    # Summary plot
    fig, ax = plt.subplots(figsize=(10, 6))
    
    features = list(outlier_summary.keys())
    outlier_pcts = [outlier_summary[f]['outlier_pct'] for f in features]
    
    bars = ax.bar(features, outlier_pcts, color='coral', edgecolor='darkred', linewidth=2)
    
    # Highlight high outlier features
    for i, pct in enumerate(outlier_pcts):
        if pct > 1.0:  # More than 1% outliers
            bars[i].set_color('red')
    
    ax.set_title('Outlier Percentage by Feature', fontsize=14, fontweight='bold')
    ax.set_ylabel('Outlier %')
    ax.set_xlabel('Feature')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for bar, pct in zip(bars, outlier_pcts):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{pct:.2f}%', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    return outlier_summary


# Practical example: Using outlier detection for data cleaning
def clean_feature_outliers(df, feature_col, outlier_info, method='cap'):
    """
    Clean outliers from a feature using different methods.
    """
    # Get outlier timestamps
    outlier_timestamps = outlier_info['outlier_timestamps']
    
    # Create copy
    df_clean = df.copy()
    
    if method == 'remove':
        # Remove rows with outliers
        mask = ~df_clean['__index_level_0__'].isin(outlier_timestamps)
        df_clean = df_clean[mask]
        print(f"Removed {len(outlier_timestamps)} rows with outliers")
        
    elif method == 'cap':
        # Cap outliers at 99th percentile
        lower_cap = df_clean[feature_col].quantile(0.01)
        upper_cap = df_clean[feature_col].quantile(0.99)
        
        df_clean[feature_col] = df_clean[feature_col].clip(lower=lower_cap, upper=upper_cap)
        print(f"Capped {feature_col} to range [{lower_cap:.2f}, {upper_cap:.2f}]")
        
    elif method == 'interpolate':
        # Replace outliers with interpolated values
        outlier_mask = df_clean['__index_level_0__'].isin(outlier_timestamps)
        df_clean.loc[outlier_mask, feature_col] = np.nan
        df_clean[feature_col] = df_clean[feature_col].interpolate(method='linear')
        print(f"Interpolated {len(outlier_timestamps)} outlier values")
    
    return df_clean


# Define your feature list

X_FEATURES = selected_features

# Example usage
if __name__ == "__main__":
    # Example 1: Single feature analysis
    data_path = '/kaggle/input/drw-crypto-market-prediction/train.parquet'
    df = pd.read_parquet(data_path, columns=['__index_level_0__', 'volume', 'label'])
    X_train_reset = X_train.reset_index(drop=True)
#    X_train_reset.index = df.index[:len(X_train)]  # Match df's index
    df = pd.concat([df,X_train_reset])
    if '__index_level_0__' not in df.columns:
        df['__index_level_0__'] = pd.date_range('2023-03-01', periods=len(df), freq='T')
    print(X_train_reset)
    # Detect outliers in volume
    outlier_info, model, forecast = simple_prophet_outlier_detection(df, 'volume')

    print(X_FEATURES)
    # Example 2: Multiple features
    outlier_summary = detect_outliers_multiple_features(
        data_path,
        X_FEATURES
    )
    
    # Example 3: Clean the data
    df_clean = df.copy()
    for feat in X_FEATURES:
        if feat in outlier_summary:
            df_clean = clean_feature_outliers(df_clean, feat, outlier_summary[feat], method='cap')
    print("\nOutlier detection complete!")

In [ ]:
!pip install koolbox scikit-learn==1.5.2

In [ ]:
from koolbox import Trainer
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from scipy.stats import pearsonr as pr
from xgboost import XGBRegressor

In [ ]:
def pearsonr(y_true, y_pred):
    return pr(y_true, y_pred)[0]

In [ ]:
lgbm_params = {
    "boosting_type": "gbdt",
    "colsample_bytree": 0.5625888953382505,
    "learning_rate": 0.029312951475451557,
    "min_child_samples": 63,
    "min_child_weight": 0.11456572852335424,
    "n_estimators": 126,
    "n_jobs": -1,
    "num_leaves": 37,
    "random_state": 42,
    "reg_alpha": 85.2476527854083,
    "reg_lambda": 99.38305361388907,
    "subsample": 0.450669817684892,
    "verbose": -1
}

lgbm_goss_params = {
    "boosting_type": "goss",
    "colsample_bytree": 0.34695458228489784,
    "learning_rate": 0.031023014900595287,
    "min_child_samples": 30,
    "min_child_weight": 0.4727729225033618,
    "n_estimators": 220,
    "n_jobs": -1,
    "num_leaves": 58,
    "random_state": 42,
    "reg_alpha": 38.665994901468224,
    "reg_lambda": 92.76991677464294,
    "subsample": 0.4810891284493255,
    "verbose": -1
}

xgb_params = {
    "colsample_bylevel": 0.4778015829774066,
    "colsample_bynode": 0.362764358742407,
    "colsample_bytree": 0.7107423488010493,
    "gamma": 1.7094857725240398,
    "learning_rate": 0.02213323588455387,
    "max_depth": 20,
    "max_leaves": 12,
    "min_child_weight": 16,
    "n_estimators": 1667,
    "n_jobs": -1,
    "random_state": 42,
    "reg_alpha": 39.352415706891264,
    "reg_lambda": 75.44843704068275,
    "subsample": 0.06566669853471274,
    "verbosity": 0
}

histgb_params = {
    "max_iter": 500,
    "learning_rate": 0.05,
    "max_depth": 10,
    "random_state": 42
}

catboost_params = {
    "iterations": 1000,
    "learning_rate": 0.02,
    "depth": 10,
    "l2_leaf_reg": 3,
    "bootstrap_type": "Bayesian",
    "bagging_temperature": 1.0,
    "loss_function": "RMSE",
    "eval_metric": "RMSE",
    "early_stopping_rounds": 100,
    "random_seed": 42,
    "task_type": "GPU",
    "verbose": 100
}

In [ ]:
fold_scores = {}
overall_scores = {}

oof_preds = {}
test_preds = {}

In [ ]:
class CFG:
    train_path = "/kaggle/input/drw-crypto-market-prediction/train.parquet"
    test_path = "/kaggle/input/drw-crypto-market-prediction/test.parquet"
    sample_sub_path = "/kaggle/input/drw-crypto-market-prediction/sample_submission.csv"

    target = "label"
    n_folds = 5
    seed = 42

    run_optuna = True
    n_optuna_trials = 500

In [ ]:
train = df_clean
# For each column, replace NaN with median for robustness
for col in df_clean.columns:
    if df_clean[col].isna().any():
        median_val = df_clean[col].median()
        df_clean[col] = df_clean[col].fillna( 0)

In [ ]:
X = train.drop(CFG.target, axis=1)
y = train[CFG.target]

In [ ]:
X = X.drop(columns=["volume"])

In [ ]:
X=X.reset_index(drop=True) 

In [ ]:
X=X.drop(columns=['__index_level_0__'])

In [ ]:
X= X[525886:]
y = y[:525886]

# LightGBM (gbdt)

In [ ]:
lgbm_trainer = Trainer(
    LGBMRegressor(**lgbm_params),
    cv=KFold(n_splits=5, shuffle=False),
    metric=pearsonr,
    task="regression",
    metric_precision=6
)

lgbm_trainer.fit(X,y)

fold_scores["LightGBM (gbdt)"] = lgbm_trainer.fold_scores
overall_scores["LightGBM (gbdt)"] = [pearsonr(lgbm_trainer.oof_preds, y)]
oof_preds["LightGBM (gbdt)"] = lgbm_trainer.oof_preds
test_preds["LightGBM (gbdt)"] = lgbm_trainer.predict(X_test)

# LightGBM (goss) 

In [ ]:
lgbm_goss_trainer = Trainer(
    LGBMRegressor(**lgbm_goss_params),
    cv=KFold(n_splits=5, shuffle=False),
    metric=pearsonr,
    task="regression",
    metric_precision=6
)

lgbm_goss_trainer.fit(X,y)

fold_scores["LightGBM (goss)"] = lgbm_goss_trainer.fold_scores
overall_scores["LightGBM (goss)"] = [pearsonr(lgbm_goss_trainer.oof_preds, y)]
oof_preds["LightGBM (goss)"] = lgbm_goss_trainer.oof_preds
test_preds["LightGBM (goss)"] = lgbm_goss_trainer.predict(X_test)

# XGBoost

In [ ]:
xgb_trainer = Trainer(
    XGBRegressor(**xgb_params),
    cv=KFold(n_splits=5, shuffle=False),
    metric=pearsonr,
    task="regression",
    metric_precision=6
)

xgb_trainer.fit(X,y)

fold_scores["XGBoost"] = xgb_trainer.fold_scores
overall_scores["XGBoost"] = [pearsonr(xgb_trainer.oof_preds, y)]
oof_preds["XGBoost"] = xgb_trainer.oof_preds
test_preds["XGBoost"] = xgb_trainer.predict(X_test)

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

histgb_params = {
    "max_iter": 500,
    "learning_rate": 0.05,
    "max_depth": 10,
    "random_state": 42
}

histgb_trainer = Trainer(
    HistGradientBoostingRegressor(**histgb_params),
    cv=KFold(n_splits=5, shuffle=False),
    metric=pearsonr,
    task="regression",
    metric_precision=6
)

histgb_trainer.fit(X, y)
fold_scores["HistGB"] = histgb_trainer.fold_scores
overall_scores["HistGB"] = [pearsonr(histgb_trainer.oof_preds, y)]
oof_preds["HistGB"] = histgb_trainer.oof_preds
test_preds["HistGB"] = histgb_trainer.predict(X_test)



In [ ]:
def plot_weights(weights, title):
    sorted_indices = np.argsort(weights[0])[::-1]
    sorted_coeffs = np.array(weights[0])[sorted_indices]
    sorted_model_names = np.array(list(oof_preds.keys()))[sorted_indices]

    plt.figure(figsize=(10, weights.shape[1] * 0.5))
    ax = sns.barplot(x=sorted_coeffs, y=sorted_model_names, palette="RdYlGn_r")

    for i, (value, name) in enumerate(zip(sorted_coeffs, sorted_model_names)):
        if value >= 0:
            ax.text(value, i, f"{value:.3f}", va="center", ha="left", color="black")
        else:
            ax.text(value, i, f"{value:.3f}", va="center", ha="right", color="black")

    xlim = ax.get_xlim()
    ax.set_xlim(xlim[0] - 0.1 * abs(xlim[0]), xlim[1] + 0.1 * abs(xlim[1]))

    plt.title(title)
    plt.xlabel("")
    plt.ylabel("")
    plt.tight_layout()
    plt.show()

In [ ]:
X = pd.DataFrame(oof_preds)
X_test = pd.DataFrame(test_preds)

In [ ]:
import joblib
joblib.dump(X, "oof_preds.pkl")
joblib.dump(X_test, "test_preds.pkl")

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin
import tensorflow as tf
import numpy as np

class AutoEncoderMLP(BaseEstimator, RegressorMixin):
    def __init__(self, num_columns, hidden_units, dropout_rates, lr=1e-3):
        self.num_columns = num_columns
        self.hidden_units = hidden_units
        self.dropout_rates = dropout_rates
        self.lr = lr
        self.model = self._build_model()
    
    def _build_model(self):
        inp = tf.keras.layers.Input(shape=(self.num_columns,))
        x0 = tf.keras.layers.BatchNormalization()(inp)

        encoder = tf.keras.layers.GaussianNoise(self.dropout_rates[0])(x0)
        encoder = tf.keras.layers.Dense(self.hidden_units[0])(encoder)
        encoder = tf.keras.layers.BatchNormalization()(encoder)
        encoder = tf.keras.layers.Activation('swish')(encoder)

        decoder = tf.keras.layers.Dropout(self.dropout_rates[1])(encoder)
        decoder = tf.keras.layers.Dense(self.num_columns, name='decoder')(decoder)

        x_reg = tf.keras.layers.Dense(self.hidden_units[1])(encoder)
        x_reg = tf.keras.layers.BatchNormalization()(x_reg)
        x_reg = tf.keras.layers.Activation('swish')(x_reg)
        x_reg = tf.keras.layers.Dropout(self.dropout_rates[2])(x_reg)

        out_reg = tf.keras.layers.Dense(1, activation='linear', name='target')(x_reg)

        model = tf.keras.models.Model(inputs=inp, outputs=[decoder, out_reg])
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.lr),
            loss={"decoder": tf.keras.losses.MeanSquaredError(),
                  "target": tf.keras.losses.MeanSquaredError()},
            loss_weights={"decoder": 0.3, "target": 1.0}
        )
        return model

    def fit(self, X, y):
        self.model.fit(
            X, {"decoder": X, "target": y},
            epochs=50,
            batch_size=8192,
            validation_split=0.2,
            callbacks=[
                tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
                tf.keras.callbacks.ReduceLROnPlateau(patience=5)
            ],
            verbose=0
        )
        return self

    def predict(self, X):
        _, y_pred = self.model.predict(X, verbose=0)
        return y_pred.flatten()

In [ ]:
ae_model = AutoEncoderMLP(
    num_columns=X.shape[1],
    hidden_units=[128, 128],
    dropout_rates=[0.05, 0.1, 0.2],
    lr=1e-3
)


ae_trainer = Trainer(
    ae_model,
    cv=KFold(n_splits=5, shuffle=False),
    metric=pearsonr,
    task="regression",
    metric_precision=6
)

ae_trainer.fit(X, y)

fold_scores["ae"] = ae_trainer.fold_scores
overall_scores["ae"] = [pearsonr(ae_trainer.oof_preds, y)]
oof_preds["ae"] = ae_trainer.oof_preds
test_preds["ae"] = ae_trainer.predict(X_test)




In [ ]:
test_preds["XGBoost"]

In [ ]:
sub = pd.read_csv(CFG.sample_sub_path)
sub["prediction"] = test_preds["XGBoost"]
sub.to_csv(f"sub_xg.csv", index=False)
sub.head()

In [ ]:
overall_scores

In [ ]:
scores = pd.DataFrame(fold_scores)
mean_scores = scores.mean().sort_values(ascending=False)
order = scores.mean().sort_values(ascending=False).index.tolist()

min_score = mean_scores.min()
max_score = mean_scores.max()
padding = (max_score - min_score) * 0.5
lower_limit = min_score - padding
upper_limit = max_score + padding

fig, axs = plt.subplots(1, 2, figsize=(15, scores.shape[1] * 0.5))

boxplot = sns.boxplot(data=scores, order=order, ax=axs[0], orient="h", color="grey")
axs[0].set_title(f"Fold Score")
axs[0].set_xlabel("")
axs[0].set_ylabel("")

barplot = sns.barplot(x=mean_scores.values, y=mean_scores.index, ax=axs[1], color="grey")
axs[1].set_title(f"Average Score")
axs[1].set_xlabel("")
axs[1].set_xlim(left=lower_limit, right=upper_limit)
axs[1].set_ylabel("")

for i, (score, model) in enumerate(zip(mean_scores.values, mean_scores.index)):
    color = "cyan" if "ensemble" in model.lower() else "grey"
    barplot.patches[i].set_facecolor(color)
    boxplot.patches[i].set_facecolor(color)
    barplot.text(score, i, round(score, 6), va="center")

plt.tight_layout()
plt.show()

In [ ]:
X